In [5]:
%matplotlib inline

import os
import numpy as np
from collections import namedtuple
from numpy import linalg as LA
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
import pandas as pd

seq = '00'
KITTI_HOME = '/home/kreimer/KITTI'
seq_dir = os.path.join(KITTI_HOME, 'dataset', 'sequences', seq)
image_0 = os.path.join(seq_dir, 'image_0')
calib_file = os.path.join(KITTI_HOME, 'dataset', 'sequences', seq, 'calib.txt')

with open(calib_file, 'r') as f:
    for line in f:
        line = [float(v) for v in line.split()[1:] ]
        K = np.array(line).reshape(3,4)
        break

Intrinsic = namedtuple('intrinsic', ['f', 'pp'])
intr = Intrinsic(f = K[0,0], pp=K[range(2),2])

def read_scales(file):
    scales = []
    with open(file, 'r') as f:
        for line in f:
            pose = np.array([float(val) for val in line.split()]).reshape(3,4)
            pose = np.vstack((pose, [0, 0, 0, 1]))
            try:
                delta = np.dot(LA.inv(ppose), pose)
            except NameError:
                ppose = pose
                continue

            scale = LA.norm(delta[:3,3])
            scales.append(scale)

            ppose = pose
    return scales

def read_image(path, mask, i):
    file_path = os.path.join(path, mask % i)
    pil_image = Image.open(file_path).convert('RGB') 
    cv_image = np.array(pil_image) 
    cv_image = cv_image[:, :, ::-1].copy()
    gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    return gray

def memory_usage_psutil():
    # return the memory usage in MB
    import psutil
    process = psutil.Process(os.getpid())
    mem = process.memory_info()[0] / float(2 ** 20)
    return mem

scales = read_scales('data/%s.txt' % seq)
scales = np.array(scales)

print('Memory usage before:', memory_usage_psutil(), 'MiB')

sequences = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']
seq = '04'

scales = read_scales('data/%s.txt' % seq)
scales = np.array(scales)
num_frames = scales.shape[0]

data = np.zeros([268, 376*2, 1241, 1], dtype=np.float32)
for i in range(num_frames):
    image = read_image(image_0, '%06d.png', i+1)
    if i<num_frames-1:
        data[i,:376,:,0] = image
    if i>0:
        data[i-1,376:,:,0] = image

print('Memory usage after reading the images:', memory_usage_psutil(), 'MiB')

Memory usage before: 1133.98046875 MiB
Memory usage after reading the images: 1134.1875 MiB


In [6]:
print('scales size: ', scales.shape)
print('images size: ', data.shape, 'dtype', data.dtype)

image_height, image_width = data.shape[1:3]
print('image height', image_height, 'image width', image_width)

scales size:  (269,)
images size:  (268, 752, 1241, 1) dtype float32
image height 752 image width 1241


In [42]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

batch_size = 16
depth = 16 # the number of output feature maps
num_hidden = 64
num_labels = 1
num_channels = 1
kernel_size = 5

graph = tf.Graph()
with graph.as_default():
  # Input data.
  train_data = tf.placeholder(tf.float32, shape=(batch_size, image_height, image_width, num_channels))
  y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  # Variables.
  weights1a = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, num_channels, depth], stddev=0.1))
  biases1a = tf.Variable(tf.zeros([depth]))

  weights2 = tf.Variable(tf.truncated_normal([batch_size, image_height//4, image_width//2, depth], stddev=0.1))
  #biases2 = tf.Variable(tf.constant(1.0, shape=[image_width*image_height//2]))
  
  # Model.
  def model(batch):
    images1, images2 = batch[:,:376,:,0], batch[:,376:,:,0]

    images1 = tf.reshape(images1, [16, 376, 1241, 1])
    images2 = tf.reshape(images2, [16, 376, 1241, 1])
    
    # convolutional layers that share weights: siamese architecture of LeCun.
    print(images1.get_shape())
    conv1a = tf.nn.conv2d(images1, weights1a, [1,1,1,1], padding='SAME')
    print(conv1a)
    hidden1a = tf.nn.relu(conv1a + biases1a)
    h_pool1a = tf.nn.max_pool(hidden1a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    conv1b = tf.nn.conv2d(images2, weights1a, [1,1,1,1], padding='SAME')
    hidden1b = tf.nn.relu(conv1b + biases1a)
    h_pool1b = tf.nn.max_pool(hidden1b, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    reshape1a = tf.reshape(h_pool1a, [batch_size, -1])
    reshape1b = tf.reshape(h_pool1b, [batch_size, -1])
    
    dim = reshape1a.get_shape()[1].value
    weights = tf.Variable(tf.truncated_normal([2*dim, 1], stddev=0.1))
    biases = tf.Variable(tf.constant(0.1, shape=[1]))

    flat = tf.concat(1, [reshape1a, reshape1b])
    print(flat)
    local1 = tf.nn.relu(tf.matmul(flat, weights) + biases)

    return local1
  
  # Training computation.
  y_ = model(train_data)
  loss = tf.reduce_sum(tf.square(y_ - y))
 
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

num_steps = 10

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    print(scales.shape)
    offset = (step * batch_size) % (scales.shape[0] - batch_size)
    
    batch_data = data[offset:(offset + batch_size), :, :, :]
    batch_labels = np.reshape(scales[offset:(offset + batch_size)], [batch_size, 1])
    
    feed_dict = {train_data : batch_data, y : batch_labels}
    
    session.run([optimizer, loss], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, loss))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

(16, 376, 1241, 1)
Tensor("Conv2D:0", shape=(16, 376, 1241, 16), dtype=float32)
Tensor("concat:0", shape=(16, 3735936), dtype=float32)
Initialized
(269,)
(269,)
(269,)
(269,)
(269,)
(269,)
(269,)
(269,)
(269,)
(269,)


KeyboardInterrupt: 

In [ ]:
print(df.columns)